# Homework

You need to train the model and get score as big as you can

Dataset: https://disk.yandex.ru/d/gcGKZ_KsXZN4VA

### Prepare part

In [1]:
import logging
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import tqdm.notebook as tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torch_data
import torchaudio
import urllib

from IPython.display import clear_output

%matplotlib inline

In [8]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/gcGKZ_KsXZN4VA'
final_url = base_url + urllib.parse.urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
!wget -O data.tar.gz "{download_url}"
!tar -xf data.tar.gz

--2024-11-28 03:58:40--  https://downloader.disk.yandex.ru/disk/00da1982ec85dee75f069c1fe0a8c6f7e5449a6753cd44ac21861925a836780d/6748932f/gtj3WQiuHGabqHv6W0pVHEXbZdC1bwqiumxRXdqgzpJ223zLuKxFz6bilpTWCZ6rE1Zoh7DdQ82OIQqL0xL28A%3D%3D?uid=0&filename=ml_trains.tar.gz&disposition=attachment&hash=ER8JmpAegW%2B8Cl46xAlRyrF9NAw2fVICg53vs0t17nID28omurL9sCwKmDpfdG9qq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fx-gzip&owner_uid=163052607&fsize=10315472612&hid=05181040c34ed1b1730124aef7dbd3fc&media_type=compressed&tknv=v2
Resolving downloader.disk.yandex.ru (downloader.disk.yandex.ru)... 77.88.21.127, 2a02:6b8::2:127
Connecting to downloader.disk.yandex.ru (downloader.disk.yandex.ru)|77.88.21.127|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s315vla.storage.yandex.net/rdisk/00da1982ec85dee75f069c1fe0a8c6f7e5449a6753cd44ac21861925a836780d/6748932f/gtj3WQiuHGabqHv6W0pVHEXbZdC1bwqiumxRXdqgzpJ223zLuKxFz6bilpTWCZ6rE1Zoh7DdQ82OIQqL0xL28A==?uid

In [3]:
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") # also you can use "cuda" for gpu and "mps" for apple silicon
DATADIR = 'ml_trains_data'
LOADER_WORKERS = 0

In [4]:
DEVICE

device(type='cuda', index=1)

In [5]:
from PIL import Image
from torchvision import transforms
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split_part: str, datadir: str, feats: nn.Module, spectrogram_dir: str,
                 read_labels=True, apply_augmentation=False):
        super().__init__()
        self.spectrogram_dir = spectrogram_dir
        os.makedirs(spectrogram_dir, exist_ok=True) 
        
        data = pd.read_csv(os.path.join(datadir, f'{split_part}.tsv'), sep='\t')
        labels = {key: idx for idx, key in enumerate(sorted(set(data.label.values)))} if read_labels else None
        self._idx_to_label = {idx: key for key, idx in labels.items()} if labels else {}
        self._classes = len(labels) if labels else 0
        self._feats = feats
        self._pathes = []
        self._labels = []

        for _, row in data.iterrows():
            path = os.path.join(datadir, 'clips', f'{row.YTID}.flac')
            if os.path.exists(path):
                self._pathes.append(path)
                self._labels.append(labels[row.label] if read_labels else 0)

                
                spec_path = os.path.join(spectrogram_dir, f"{row.YTID}.png")
                if not os.path.exists(spec_path):  
                    audio = Audio(path)
                    audio.save_spectrogram(spec_path)
        self.transform = transforms.Compose([
            transforms.ToTensor(),  # Преобразование в тензор (C, H, W)
        ])
    def __getitem__(self, index):
        yt_id = os.path.splitext(os.path.basename(self._pathes[index]))[0]  # Извлечение YTID
        spec_path = os.path.join(self.spectrogram_dir, f"{yt_id}.png")
        if not os.path.exists(spec_path):
            raise FileNotFoundError(f"Спектрограмма не найдена: {spec_path}")
        spectrogram = Image.open(spec_path).convert("RGB")
        spectrogram = self.transform(spectrogram)
        label = self._labels[index]
        return spectrogram, label

    def __len__(self):
        return len(self._pathes)

    def classes(self):
        return self._classes
SPECTROGRAM_DPI = 90 
DEFAULT_SAMPLE_RATE = 44100
DEFAULT_HOPE_LENGHT = 1024

def apply_effect(waveform, sample_rate, effect):
    effector = torchaudio.io.AudioEffector(effect=effect)
    return effector.apply(waveform, sample_rate)

def collate_fn(batch):
    max_length = max(item[0].shape[1] for item in batch)
    max_width = max(item[0].shape[2] for item in batch)
    X = torch.zeros((len(batch), 3, max_length, max_width))
    for idx, item in enumerate(batch):
        c, h, w = item[0].shape
        X[idx, :, :h, :w] = item[0]  
    
 
    targets = torch.tensor([item[1] for item in batch], dtype=torch.long)
    return X, targets
class Audio:
    def __init__(self, filepath_, hop_length=DEFAULT_HOPE_LENGHT, sample_rate=DEFAULT_SAMPLE_RATE):
        self.hop_length = hop_length
        self.sample_rate = sample_rate
        self.waveform, self.sample_rate = torchaudio.load(filepath_)

    def save_spectrogram(self, path, dpi=SPECTROGRAM_DPI):
        waveform = self.waveform.numpy()
        plt.figure(figsize=(10, 4))
        plt.specgram(waveform[0], Fs=self.sample_rate)
        plt.axis('off')
        plt.savefig(path, dpi=dpi, bbox_inches='tight', pad_inches=0)
        plt.close()  

### Feats part

In [6]:
# Try to use LogMelSpectrogram or MFCC to achive better score
class LogMelSpectrogram(nn.Module):
    pass

# FBANK 40 by default, but you can choose something else
FEATS = 40
spectrogram_dir = "./spectrograms"
transform = torchaudio.transforms.MelSpectrogram(n_mels=FEATS)
trainset = Dataset('train', 'ml_trains_data', transform, apply_augmentation=False,spectrogram_dir=spectrogram_dir)
testset = Dataset('eval', 'ml_trains_data', transform, read_labels=False, apply_augmentation=False, spectrogram_dir=spectrogram_dir)
N_CLASSES = trainset.classes()
assert N_CLASSES == 30

In [40]:
print(torch.__version__)
print(torchaudio.__version__)

2.5.1
2.5.1


In [7]:
print(f"Total test examples loaded: {len(testset)}")

Total test examples loaded: 3000


### Eval part

Write balanced accuracy:
$$BAcc = \frac{1}{classes}\sum_{c = 1}^{classes} \frac{\sum_i^n I(y_i = p_i = c)}{\sum_i^n I(y_i = c)}$$

Where:
- $y_i$ -- target class for $i$ element
- $p_i$ -- predicted class for $i$ element

In [8]:
# Get list of pairs (target_class, predicted_class)
from collections import defaultdict
def balanced_accuracy(items: list[tuple[int, int]], classes=N_CLASSES) -> float:
    true_counts = defaultdict(int)  # Счётчик истинных классов
    correct_counts = defaultdict(int)  # Счётчик верных предсказаний
    
    # Подсчёт по всем элементам
    for y_true, y_pred in items:
        true_counts[y_true] += 1
        if y_true == y_pred:
            correct_counts[y_true] += 1
    
    # Вычисление сбалансированной точности
    class_accuracies = []
    for c in range(classes):
        if true_counts[c] > 0:
            class_accuracies.append(correct_counts[c] / true_counts[c])
        else:
            class_accuracies.append(0.0)  # Если класс отсутствует
    
    return sum(class_accuracies) / classes

In [9]:
assert np.isclose(balanced_accuracy([(0, 0), (0, 0), (1, 1)], 2), 1.0)
assert np.isclose(balanced_accuracy([(0, 1), (1, 0)], 2), 0.0)
assert np.isclose(balanced_accuracy([(0, 0), (0, 0), (1, 0)], 2), 0.5)
assert np.isclose(balanced_accuracy([(0, 0), (1, 1), (0, 0), (0, 0), (1, 0), (0, 1)], 2), 0.625)
assert np.isclose(balanced_accuracy([(1, 1), (0, 1), (2, 2)], 3), 0.66666666666666)

### Train part
Train some model with as big balanced accuracy as you can

You can train any model you want. The only limitation is that it must be trained from scratch on the data provided in the task. For example you can choose model from:
- DNN
- CNN 1d
- CNN 2d
- Transformer
- RNN
- mixes of given models

Hints:
- No need to train large models for this task. 10 million parameters is more than you need.
- Watch to overfitting, try to add Augmentation, Dropout, BatchNorm, L1/L2-Regulatization or something else.
- Use poolings or strides to reduce time-dimenstion. It is better to reduce the dimension gradually rather than at the end.
- Try different features (mel-spec, log-mel-spec, mfcc)

In [11]:
def stage(
    model: nn.Module,
    data: torch_data.Dataset,
    opt: optim.Optimizer,
    batch_size: int = 256,
    train: bool = True
):
    loader = torch_data.DataLoader(
        data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=LOADER_WORKERS,
        collate_fn=collate_fn,
    )
    if train:
        model.train()
    else:
        model.eval()
    loss_sum, batches = 0.0, 0
    pred_pairs = []
    for X, Y in tqdm.tqdm(loader):
        pred = model.forward(X.to(DEVICE))
        loss = F.cross_entropy(pred.squeeze(), Y.squeeze().to(DEVICE))
        if train:
            opt.zero_grad()
            loss.backward()
            opt.step()
        loss_sum += loss.item()
        batches += 1
        with torch.no_grad():
            pred_pairs.extend(zip(
                Y.data.numpy().reshape(-1),
                torch.argmax(pred, dim=1).cpu().data.numpy().reshape(-1)
            ))
    return loss_sum / batches, balanced_accuracy(pred_pairs)


def train(
    model: nn.Module,
    opt,
    batch_size: int = 256,
    epochs: int = 70,
):
    best_valid_loss = float("inf")
    train_data, val_data = torch.utils.data.random_split(trainset, (0.8, 0.2), generator=torch.Generator().manual_seed(42))
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    for epoch in range(epochs):
        train_loss, train_acc = stage(model, train_data, opt, batch_size=batch_size)
        val_loss, val_acc = stage(model, val_data, opt, batch_size=batch_size, train=False)
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        if val_loss < best_valid_loss:
            print('best', epoch)
            best_valid_loss = val_loss
            torch.save(model.state_dict(), "tut-model_large.pt")
            
        
        #fig, axis = plt.subplots(1, 2, figsize=(15, 7))
        #axis[0].plot(np.arange(1, epoch + 2), train_losses, label='train')
        #axis[0].plot(np.arange(1, epoch + 2), val_losses, label='val')
        #axis[1].plot(np.arange(1, epoch + 2), train_accs, label='train')
        #axis[1].plot(np.arange(1, epoch + 2), val_accs, label='val')
        #axis[0].set(xlabel='epoch', ylabel='CE Loss')
        #axis[1].set(xlabel='epoch', ylabel='Accuracy')
        #fig.legend()
        #plt.show()
        print(f'Epoch {epoch + 1}.')
        print(f'Train loss {train_loss}. Train accuracy {train_acc}.')
        print(f'Test loss {val_loss}. Test accuracy {val_acc}')
    print(f'Epoch {epoch + 1}.')
    print(f'Train loss {train_loss}. Train accuracy {train_acc}.')
    print(f'Test loss {val_loss}. Test accuracy {val_acc}')

In [58]:
import torch
import torch.nn as nn
DATASET_PATH = "./spectrograms"
IMAGE_SIZE = (1024, 1024) # image size
CHANNEL_COUNT = 3 
class NeuralNetworkV4(nn.Module):
    def __init__(self, n_classes=N_CLASSES):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)  
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)  
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)  
        self.conv4 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)

        self.bn1 = nn.BatchNorm2d(8)
        self.bn2 = nn.BatchNorm2d(16)
        self.bn3 = nn.BatchNorm2d(32)
        self.bn4 = nn.BatchNorm2d(64)
        

        self.pooling = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(2816, 1024)  
        self.fc2 = nn.Linear(1024, 512)  
        self.fc3 = nn.Linear(512, n_classes)
        

        self.relu = nn.ELU()
        self.dropout = nn.Dropout(0.3)  
    
    def forward(self, x: torch.Tensor):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pooling(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pooling(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pooling(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pooling(x)
        x = self.flatten(x)
        #print(x.shape)
        x = self.dropout(self.relu(self.fc1(x)))
        #print(x.shape)
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x


In [59]:
model = NeuralNetworkV4().to(DEVICE)

In [60]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 3,449,566 trainable parameters


In [ ]:
LEARNING_RATE = 0.0001
GRADIENT_MOMENTUM = 0.90
opt = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=GRADIENT_MOMENTUM)
train(model, opt)

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

best 0
Epoch 1.
Train loss 1.9824029308993643. Train accuracy 0.41192243134247203.
Test loss 2.167570363907587. Test accuracy 0.3533237788795827


  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

...

Epoch 58.
Train loss 1.8815610786763632. Train accuracy 0.4364781981298208.
Test loss 2.129889476866949. Test accuracy 0.36638059180826493


  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

Epoch 59.
Train loss 1.8800535318328113. Train accuracy 0.43728833160580116.
Test loss 2.125988636698042. Test accuracy 0.36915153958780356


  0%|          | 0/82 [00:00<?, ?it/s]

In [63]:
model.load_state_dict(torch.load("tut-model_large.pt"))

/tmp/ipykernel_2751310/3825035237.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("tut-model_large.pt"))


<All keys matched successfully>

### Prediction part

Prepare result file and send it to Yandex.Contest

In [64]:
result = []
for idx, path in enumerate(testset._pathes):
    feats, _ = testset[idx]
    predict = torch.argmax(model.forward(feats.unsqueeze(0).to(DEVICE)), dim=1).item()
    ytid = os.path.basename(path).replace('.flac', '')  
    result.append((ytid, trainset._idx_to_label[predict]))
pd.DataFrame(result, columns=['YTID', 'label']).to_csv('result.tsv', sep='\t', index=False)